In [1]:
# 주의! ray를 tensorflow보다 먼저 import하면 오류가 발생할 수 있습니다
import io, json, os, math

import tensorflow as tf
from tensorflow.keras.layers import Add, Concatenate, Lambda
from tensorflow.keras.layers import Input, Conv2D, ReLU, MaxPool2D
from tensorflow.keras.layers import UpSampling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
import ray

import time
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

PROJECT_PATH = os.getenv('HOME') + '/aiffel/node_data/GD08/mpii'
MODEL_PATH = os.getenv('HOME') + '/aiffel/model_weight/GD08'
IMAGE_PATH = os.path.join(PROJECT_PATH, 'images')
TFRECORD_PATH = os.path.join(PROJECT_PATH, 'tfrecords_mpii')
TRAIN_JSON = os.path.join(PROJECT_PATH, 'mpii_human_pose_v1_u12_2', 'train.json')
VALID_JSON = os.path.join(PROJECT_PATH, 'mpii_human_pose_v1_u12_2', 'validation.json')

### Label 변환 함수

In [2]:
def parse_tfexample(example):
    image_feature_description = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/depth': tf.io.FixedLenFeature([], tf.int64),
        'image/object/parts/x': tf.io.VarLenFeature(tf.int64),
        'image/object/parts/y': tf.io.VarLenFeature(tf.int64),
        'image/object/parts/v': tf.io.VarLenFeature(tf.int64),
        'image/object/center/x': tf.io.FixedLenFeature([], tf.int64),
        'image/object/center/y': tf.io.FixedLenFeature([], tf.int64),
        'image/object/scale': tf.io.FixedLenFeature([], tf.float32),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
    }
    return tf.io.parse_single_example(example, image_feature_description)

In [3]:
def crop_roi(image, features, margin=0.2):
    img_shape = tf.shape(image)
    img_height = img_shape[0]
    img_width = img_shape[1]
    img_depth = img_shape[2]

    keypoint_x = tf.cast(tf.sparse.to_dense(features['image/object/parts/x']), dtype=tf.int32)
    keypoint_y = tf.cast(tf.sparse.to_dense(features['image/object/parts/y']), dtype=tf.int32)
    center_x = features['image/object/center/x']
    center_y = features['image/object/center/y']
    body_height = features['image/object/scale'] * 200.0

    # keypoint 중 유효한값(visible = 1) 만 사용합니다.
    masked_keypoint_x = tf.boolean_mask(keypoint_x, keypoint_x > 0)
    masked_keypoint_y = tf.boolean_mask(keypoint_y, keypoint_y > 0)

    # min, max 값을 찾습니다.
    keypoint_xmin = tf.reduce_min(masked_keypoint_x)
    keypoint_xmax = tf.reduce_max(masked_keypoint_x)
    keypoint_ymin = tf.reduce_min(masked_keypoint_y)
    keypoint_ymax = tf.reduce_max(masked_keypoint_y)

    # 높이 값을 이용해서 x, y 위치를 재조정 합니다. 박스를 정사각형으로 사용하기 위해 아래와 같이 사용합니다.
    xmin = keypoint_xmin - tf.cast(body_height * margin, dtype=tf.int32)
    xmax = keypoint_xmax + tf.cast(body_height * margin, dtype=tf.int32)
    ymin = keypoint_ymin - tf.cast(body_height * margin, dtype=tf.int32)
    ymax = keypoint_ymax + tf.cast(body_height * margin, dtype=tf.int32)

    # 이미지 크기를 벗어나는 점을 재조정 해줍니다.
    effective_xmin = xmin if xmin > 0 else 0
    effective_ymin = ymin if ymin > 0 else 0
    effective_xmax = xmax if xmax < img_width else img_width
    effective_ymax = ymax if ymax < img_height else img_height
    effective_height = effective_ymax - effective_ymin
    effective_width = effective_xmax - effective_xmin

    image = image[effective_ymin:effective_ymax, effective_xmin:effective_xmax, :]
    new_shape = tf.shape(image)
    new_height = new_shape[0]
    new_width = new_shape[1]

    effective_keypoint_x = (keypoint_x - effective_xmin) / new_width
    effective_keypoint_y = (keypoint_y - effective_ymin) / new_height

    return image, effective_keypoint_x, effective_keypoint_y

In [4]:
def generate_2d_guassian(height, width, y0, x0, visibility=2, sigma=1, scale=12):
    heatmap = tf.zeros((height, width))

    xmin = x0 - 3 * sigma
    ymin = y0 - 3 * sigma
    xmax = x0 + 3 * sigma
    ymax = y0 + 3 * sigma
    
    if xmin >= width or ymin >= height or xmax < 0 or ymax < 0 or visibility == 0:
        return heatmap

    size = 6 * sigma + 1
    x, y = tf.meshgrid(tf.range(0, 6 * sigma + 1, 1), tf.range(0, 6 * sigma + 1, 1), indexing='xy')

    center_x = size // 2
    center_y = size // 2

    gaussian_patch = tf.cast(tf.math.exp(
        -(tf.math.square(x - center_x) + tf.math.square(y - center_y)) / (tf.math.square(sigma) * 2)) * scale,
                             dtype=tf.float32)

    patch_xmin = tf.math.maximum(0, -xmin)
    patch_ymin = tf.math.maximum(0, -ymin)
    patch_xmax = tf.math.minimum(xmax, width) - xmin
    patch_ymax = tf.math.minimum(ymax, height) - ymin

    heatmap_xmin = tf.math.maximum(0, xmin)
    heatmap_ymin = tf.math.maximum(0, ymin)
    heatmap_xmax = tf.math.minimum(xmax, width)
    heatmap_ymax = tf.math.minimum(ymax, height)

    indices = tf.TensorArray(tf.int32, 1, dynamic_size=True)
    updates = tf.TensorArray(tf.float32, 1, dynamic_size=True)

    count = 0

    for j in tf.range(patch_ymin, patch_ymax):
        for i in tf.range(patch_xmin, patch_xmax):
            indices = indices.write(count, [heatmap_ymin + j, heatmap_xmin + i])
            updates = updates.write(count, gaussian_patch[j][i])
            count += 1

    heatmap = tf.tensor_scatter_nd_update(heatmap, indices.stack(), updates.stack())

    return heatmap

def make_heatmaps(features, keypoint_x, keypoint_y, heatmap_shape):
    v = tf.cast(tf.sparse.to_dense(features['image/object/parts/v']), dtype=tf.float32)
    x = tf.cast(tf.math.round(keypoint_x * heatmap_shape[0]), dtype=tf.int32)
    y = tf.cast(tf.math.round(keypoint_y * heatmap_shape[1]), dtype=tf.int32)

    num_heatmap = heatmap_shape[2]
    heatmap_array = tf.TensorArray(tf.float32, 16)

    for i in range(num_heatmap):
        gaussian = self.generate_2d_guassian(heatmap_shape[1], heatmap_shape[0], y[i], x[i], v[i])
        heatmap_array = heatmap_array.write(i, gaussian)

    heatmaps = heatmap_array.stack()
    heatmaps = tf.transpose(heatmaps, perm=[1, 2, 0])  # change to (64, 64, 16)

    return heatmaps

In [5]:
class Preprocessor(object):
    def __init__(self,
                 image_shape=(256, 256, 3),
                 heatmap_shape=(64, 64, 16),
                 is_train=False):
        self.is_train = is_train
        self.image_shape = image_shape
        self.heatmap_shape = heatmap_shape

    def __call__(self, example):
        features = self.parse_tfexample(example)
        image = tf.io.decode_jpeg(features['image/encoded'])

        if self.is_train:
            random_margin = tf.random.uniform([1], 0.1, 0.3)[0]
            image, keypoint_x, keypoint_y = self.crop_roi(image, features, margin=random_margin)
            image = tf.image.resize(image, self.image_shape[0:2])
        else:
            image, keypoint_x, keypoint_y = self.crop_roi(image, features)
            image = tf.image.resize(image, self.image_shape[0:2])

        image = tf.cast(image, tf.float32) / 127.5 - 1
        heatmaps = self.make_heatmaps(features, keypoint_x, keypoint_y, self.heatmap_shape)

        return image, heatmaps

        
    def crop_roi(self, image, features, margin=0.2):
        img_shape = tf.shape(image)
        img_height = img_shape[0]
        img_width = img_shape[1]
        img_depth = img_shape[2]

        keypoint_x = tf.cast(tf.sparse.to_dense(features['image/object/parts/x']), dtype=tf.int32)
        keypoint_y = tf.cast(tf.sparse.to_dense(features['image/object/parts/y']), dtype=tf.int32)
        center_x = features['image/object/center/x']
        center_y = features['image/object/center/y']
        body_height = features['image/object/scale'] * 200.0
        
        masked_keypoint_x = tf.boolean_mask(keypoint_x, keypoint_x > 0)
        masked_keypoint_y = tf.boolean_mask(keypoint_y, keypoint_y > 0)
        
        keypoint_xmin = tf.reduce_min(masked_keypoint_x)
        keypoint_xmax = tf.reduce_max(masked_keypoint_x)
        keypoint_ymin = tf.reduce_min(masked_keypoint_y)
        keypoint_ymax = tf.reduce_max(masked_keypoint_y)
        
        xmin = keypoint_xmin - tf.cast(body_height * margin, dtype=tf.int32)
        xmax = keypoint_xmax + tf.cast(body_height * margin, dtype=tf.int32)
        ymin = keypoint_ymin - tf.cast(body_height * margin, dtype=tf.int32)
        ymax = keypoint_ymax + tf.cast(body_height * margin, dtype=tf.int32)
        
        effective_xmin = xmin if xmin > 0 else 0
        effective_ymin = ymin if ymin > 0 else 0
        effective_xmax = xmax if xmax < img_width else img_width
        effective_ymax = ymax if ymax < img_height else img_height
        effective_height = effective_ymax - effective_ymin
        effective_width = effective_xmax - effective_xmin

        image = image[effective_ymin:effective_ymax, effective_xmin:effective_xmax, :]
        new_shape = tf.shape(image)
        new_height = new_shape[0]
        new_width = new_shape[1]
        
        effective_keypoint_x = (keypoint_x - effective_xmin) / new_width
        effective_keypoint_y = (keypoint_y - effective_ymin) / new_height
        
        return image, effective_keypoint_x, effective_keypoint_y
        
    
    def generate_2d_guassian(self, height, width, y0, x0, visibility=2, sigma=1, scale=12):
        
        heatmap = tf.zeros((height, width))

        xmin = x0 - 3 * sigma
        ymin = y0 - 3 * sigma
        xmax = x0 + 3 * sigma
        ymax = y0 + 3 * sigma

        if xmin >= width or ymin >= height or xmax < 0 or ymax <0 or visibility == 0:
            return heatmap

        size = 6 * sigma + 1
        x, y = tf.meshgrid(tf.range(0, 6*sigma+1, 1), tf.range(0, 6*sigma+1, 1), indexing='xy')

        center_x = size // 2
        center_y = size // 2

        gaussian_patch = tf.cast(tf.math.exp(-(tf.square(x - center_x) + tf.math.square(y - center_y)) / (tf.math.square(sigma) * 2)) * scale, dtype=tf.float32)

        patch_xmin = tf.math.maximum(0, -xmin)
        patch_ymin = tf.math.maximum(0, -ymin)
        patch_xmax = tf.math.minimum(xmax, width) - xmin
        patch_ymax = tf.math.minimum(ymax, height) - ymin

        heatmap_xmin = tf.math.maximum(0, xmin)
        heatmap_ymin = tf.math.maximum(0, ymin)
        heatmap_xmax = tf.math.minimum(xmax, width)
        heatmap_ymax = tf.math.minimum(ymax, height)

        indices = tf.TensorArray(tf.int32, 1, dynamic_size=True)
        updates = tf.TensorArray(tf.float32, 1, dynamic_size=True)

        count = 0

        for j in tf.range(patch_ymin, patch_ymax):
            for i in tf.range(patch_xmin, patch_xmax):
                indices = indices.write(count, [heatmap_ymin+j, heatmap_xmin+i])
                updates = updates.write(count, gaussian_patch[j][i])
                count += 1
                
        heatmap = tf.tensor_scatter_nd_update(heatmap, indices.stack(), updates.stack())

        return heatmap


    def make_heatmaps(self, features, keypoint_x, keypoint_y, heatmap_shape):
        v = tf.cast(tf.sparse.to_dense(features['image/object/parts/v']), dtype=tf.float32)
        x = tf.cast(tf.math.round(keypoint_x * heatmap_shape[0]), dtype=tf.int32)
        y = tf.cast(tf.math.round(keypoint_y * heatmap_shape[1]), dtype=tf.int32)
        
        num_heatmap = heatmap_shape[2]
        heatmap_array = tf.TensorArray(tf.float32, 16)

        for i in range(num_heatmap):
            gaussian = self.generate_2d_guassian(heatmap_shape[1], heatmap_shape[0], y[i], x[i], v[i])
            heatmap_array = heatmap_array.write(i, gaussian)
        
        heatmaps = heatmap_array.stack()
        heatmaps = tf.transpose(heatmaps, perm=[1, 2, 0]) # change to (64, 64, 16)
        
        return heatmaps

    def parse_tfexample(self, example):
        image_feature_description = {
            'image/height': tf.io.FixedLenFeature([], tf.int64),
            'image/width': tf.io.FixedLenFeature([], tf.int64),
            'image/depth': tf.io.FixedLenFeature([], tf.int64),
            'image/object/parts/x': tf.io.VarLenFeature(tf.int64),
            'image/object/parts/y': tf.io.VarLenFeature(tf.int64),
            'image/object/parts/v': tf.io.VarLenFeature(tf.int64),
            'image/object/center/x': tf.io.FixedLenFeature([], tf.int64),
            'image/object/center/y': tf.io.FixedLenFeature([], tf.int64),
            'image/object/scale': tf.io.FixedLenFeature([], tf.float32),
            'image/encoded': tf.io.FixedLenFeature([], tf.string),
            'image/filename': tf.io.FixedLenFeature([], tf.string),
        }
        return tf.io.parse_single_example(example,
                                          image_feature_description)

### 모델 구축 함수

In [6]:
def BottleneckBlock(inputs, filters, strides=1, downsample=False, name=None):
    identity = inputs
    if downsample:
        identity = Conv2D(
            filters=filters,
            kernel_size=1,
            strides=strides,
            padding='same',
            kernel_initializer='he_normal')(inputs)

    x = BatchNormalization(momentum=0.9)(inputs)
    x = ReLU()(x)
    x = Conv2D(
        filters=filters // 2,
        kernel_size=1,
        strides=1,
        padding='same',
        kernel_initializer='he_normal')(x)

    x = BatchNormalization(momentum=0.9)(x)
    x = ReLU()(x)
    x = Conv2D(
        filters=filters // 2,
        kernel_size=3,
        strides=strides,
        padding='same',
        kernel_initializer='he_normal')(x)

    x = BatchNormalization(momentum=0.9)(x)
    x = ReLU()(x)
    x = Conv2D(
        filters=filters,
        kernel_size=1,
        strides=1,
        padding='same',
        kernel_initializer='he_normal')(x)

    x = Add()([identity, x])
    return x

In [7]:
def HourglassModule(inputs, order, filters, num_residual):
    
    up1 = BottleneckBlock(inputs, filters, downsample=False)
    for i in range(num_residual):
        up1 = BottleneckBlock(up1, filters, downsample=False)

    low1 = MaxPool2D(pool_size=2, strides=2)(inputs)
    for i in range(num_residual):
        low1 = BottleneckBlock(low1, filters, downsample=False)

    low2 = low1
    if order > 1:
        low2 = HourglassModule(low1, order - 1, filters, num_residual)
    else:
        for i in range(num_residual):
            low2 = BottleneckBlock(low2, filters, downsample=False)

    low3 = low2
    for i in range(num_residual):
        low3 = BottleneckBlock(low3, filters, downsample=False)

    up2 = UpSampling2D(size=2)(low3)

    return up2 + up1

In [8]:
def LinearLayer(inputs, filters):
    x = Conv2D(
        filters=filters,
        kernel_size=1,
        strides=1,
        padding='same',
        kernel_initializer='he_normal')(inputs)
    x = BatchNormalization(momentum=0.9)(x)
    x = ReLU()(x)
    return x

In [9]:
def StackedHourglassNetwork(
        input_shape=(256, 256, 3), 
        num_stack=4, 
        num_residual=1,
        num_heatmap=16):
    
    inputs = Input(shape=input_shape)

    x = Conv2D(
        filters=64,
        kernel_size=7,
        strides=2,
        padding='same',
        kernel_initializer='he_normal')(inputs)
    x = BatchNormalization(momentum=0.9)(x)
    x = ReLU()(x)
    x = BottleneckBlock(x, 128, downsample=True)
    x = MaxPool2D(pool_size=2, strides=2)(x)
    x = BottleneckBlock(x, 128, downsample=False)
    x = BottleneckBlock(x, 256, downsample=True)

    ys = []
    for i in range(num_stack):
        x = HourglassModule(x, order=4, filters=256, num_residual=num_residual)
        for i in range(num_residual):
            x = BottleneckBlock(x, 256, downsample=False)

        x = LinearLayer(x, 256)

        y = Conv2D(
            filters=num_heatmap,
            kernel_size=1,
            strides=1,
            padding='same',
            kernel_initializer='he_normal')(x)
        ys.append(y)

        if i < num_stack - 1:
            y_intermediate_1 = Conv2D(filters=256, kernel_size=1, strides=1)(x)
            y_intermediate_2 = Conv2D(filters=256, kernel_size=1, strides=1)(y)
            x = Add()([y_intermediate_1, y_intermediate_2])

    return tf.keras.Model(inputs, ys, name='stacked_hourglass')

In [10]:
class Trainer(object):
    def __init__(self,
                 model,
                 epochs,
                 global_batch_size,
                 strategy,
                 initial_learning_rate):
        self.model = model
        self.epochs = epochs
        self.strategy = strategy
        self.global_batch_size = global_batch_size
        self.loss_object = tf.keras.losses.MeanSquaredError(
            reduction=tf.keras.losses.Reduction.NONE)
        self.optimizer = tf.keras.optimizers.Adam(
            learning_rate=initial_learning_rate)
        self.model = model

        self.current_learning_rate = initial_learning_rate
        self.last_val_loss = math.inf
        self.lowest_val_loss = math.inf
        self.patience_count = 0
        self.max_patience = 10
        self.best_model = None

    def lr_decay(self):
        if self.patience_count >= self.max_patience:
            self.current_learning_rate /= 10.0
            self.patience_count = 0
        elif self.last_val_loss == self.lowest_val_loss:
            self.patience_count = 0
        self.patience_count += 1

        self.optimizer.learning_rate = self.current_learning_rate

    def lr_decay_step(self, epoch):
        if epoch == 25 or epoch == 50 or epoch == 75:
            self.current_learning_rate /= 10.0
        self.optimizer.learning_rate = self.current_learning_rate

    def compute_loss(self, labels, outputs):
        loss = 0
        for output in outputs:
            weights = tf.cast(labels > 0, dtype=tf.float32) * 81 + 1
            loss += tf.math.reduce_mean(
                tf.math.square(labels - output) * weights) * (
                    1. / self.global_batch_size)
        return loss
    
    def compute_mAP(self, labels, outputs, thr=0.5):
        mAP = 0.0
        map_batch = tf.zeros([self.global_batch_size], tf.float32)
        
        labels = tf.cast(labels, dtype=tf.float32)
        outputs = tf.cast(outputs, dtype=tf.float32)
            
        for output in outputs:
            distances = tf.reduce_mean(tf.sqrt(tf.square(labels - output)), axis=(1, 2))
            correct_predictions = tf.cast(distances >= thr, tf.float32)
            precision = tf.reduce_mean(correct_predictions, axis=-1)
            map_batch += precision
        map_batch = map_batch/len(outputs)
        mAP = tf.reduce_mean(map_batch)
        return mAP
    
    def compute_PCKh(self, labels, outputs, thr=0.5):
        labels = tf.cast(labels, dtype=tf.float32)
        outputs = tf.cast(outputs, dtype=tf.float32)
        
        def get_coords(heatmap):
            if len(heatmap.shape) <= 3:
                heatmap = tf.expand_dims(heatmap, axis=-1)
                
            height = heatmap.shape[1]
            widht = heatmap.shape[2]
            classes = heatmap.shape[3]
            
            flattened_heatmap = tf.reshape(tensor=heatmap, shape=[self.global_batch_size, -1, classes])
            max_coords = tf.argmax(flattened_heatmap, axis=1)
            
            y = max_coords // height
            x = max_coords % widht
            
            return tf.stack([x, y], axis=-2)
        
        label_0_coords = tf.cast(tf.squeeze(get_coords(labels[..., 0]), axis=-1), tf.float32)
        label_1_coords = tf.cast(tf.squeeze(get_coords(labels[..., 1]), axis=-1), tf.float32)
        head_size = tf.sqrt(tf.reduce_sum(tf.square(label_0_coords - label_1_coords), axis=-1))
        
        predictions = []
        for class_idx in range(labels.shape[-1]):
            prediction = tf.zeros([self.global_batch_size], tf.float32)
            for output in outputs:
                label_coords = tf.cast(tf.squeeze(get_coords(labels[..., class_idx])), tf.float32)
                output_coords = tf.cast(tf.squeeze(get_coords(output[..., class_idx])), tf.float32)
                
                dist = tf.sqrt(tf.reduce_sum(tf.square(label_coords - output_coords), axis=-1))
                correct_prediction = tf.cast(dist <= thr * head_size, tf.float32)
                prediction += correct_prediction
            prediction = prediction/len(outputs)
            predictions.append(prediction)
            
        predictions = tf.stack(predictions, axis=-1)
        PCKh = tf.reduce_mean(predictions)
        return PCKh
        
    def train_step(self, inputs):
        images, labels = inputs
        with tf.GradientTape() as tape:
            outputs = self.model(images, training=True)
            loss = self.compute_loss(labels, outputs)
            
        grads = tape.gradient(
            target=loss, sources=self.model.trainable_variables)
        self.optimizer.apply_gradients(
            zip(grads, self.model.trainable_variables))
        
        mAP = self.compute_mAP(labels, outputs)
        PCKh = self.compute_PCKh(labels, outputs)
        
        return loss, mAP, PCKh
    
    def val_step(self, inputs):
        images, labels = inputs
        outputs = self.model(images, training=False)
        loss = self.compute_loss(labels, outputs)
        
        mAP = self.compute_mAP(labels, outputs)
        PCKh = self.compute_PCKh(labels, outputs)
        
        return loss, mAP, PCKh

    def run(self, train_dist_dataset, val_dist_dataset):
        @tf.function
        def distributed_train_epoch(dataset):
            tf.print('Start distributed traininng...')
            total_loss = 0.0
            total_map = 0.0
            total_pckh = 0.0
            num_train_batches = 0.0
            for one_batch in dataset:
                per_replica_loss, per_replica_map, per_replica_pckh  = self.strategy.run(
                    self.train_step, args=(one_batch, ))
                
                num_train_batches += 1
                batch_loss = self.strategy.reduce(
                    tf.distribute.ReduceOp.SUM, per_replica_loss, axis=None)
                batch_map = self.strategy.reduce(
                    tf.distribute.ReduceOp.SUM, per_replica_map, axis=None)
                batch_pckh = self.strategy.reduce(
                    tf.distribute.ReduceOp.SUM, per_replica_pckh, axis=None)
                tf.print('Trained batch', num_train_batches,
                         'batch loss', batch_loss,
                         'batch mAP', batch_map,
                         'batch PCKh', batch_pckh)
                
                total_loss += batch_loss
                total_map += batch_map
                total_pckh += batch_pckh
                
            return total_loss, total_map, total_pckh, num_train_batches

        @tf.function
        def distributed_val_epoch(dataset):
            total_loss = 0.0
            total_map = 0.0
            total_pckh = 0.0
            num_val_batches = 0.0
            for one_batch in dataset:
                per_replica_loss, per_replica_map, per_replica_pckh = self.strategy.run(
                    self.val_step, args=(one_batch, ))
                
                num_val_batches += 1
                batch_loss = self.strategy.reduce(
                    tf.distribute.ReduceOp.SUM, per_replica_loss, axis=None)
                batch_map = self.strategy.reduce(
                    tf.distribute.ReduceOp.SUM, per_replica_map, axis=None)
                batch_pckh = self.strategy.reduce(
                    tf.distribute.ReduceOp.SUM, per_replica_pckh, axis=None)
                tf.print('Validated batch', num_val_batches,
                         'batch loss', batch_loss,
                         'batch mAP', batch_map,
                         'batch PCKh', batch_pckh)
                
                if not tf.math.is_nan(batch_loss):
                    # TODO: Find out why the last validation batch loss become NaN
                    total_loss += batch_loss
                    total_map += batch_map
                    total_pckh += batch_pckh
                else:
                    num_val_batches -= 1
                    
            return total_loss, total_map, total_pckh, num_val_batches
        
        history = {
            "train_loss": [],
            "val_loss": [],
            "train_map": [],
            "val_map": [],
            "train_pckh": [],
            "val_pckh": [],
            "epoch_time": []}
        
        for epoch in range(1, self.epochs + 1):
            start_time = time.time()
            self.lr_decay()
            print('Start epoch {} with learning rate {}'.format(
                epoch, self.current_learning_rate))
            
            train_total_loss, train_total_map, train_total_pckh, num_train_batches = distributed_train_epoch(
                train_dist_dataset)
            train_loss = train_total_loss / num_train_batches
            train_map = train_total_map / num_train_batches
            train_pckh = train_total_pckh / num_train_batches
            print('Epoch {} train loss {} train mAP {} train PCKh {}'.format(epoch, train_loss, train_map, train_pckh))
            
            val_total_loss, val_total_map, val_total_pckh, num_val_batches = distributed_val_epoch(
                val_dist_dataset)
            val_loss = val_total_loss / num_val_batches
            val_map = val_total_map / num_val_batches
            val_pckh = val_total_pckh / num_val_batches
            print('Epoch {} val loss {} val mAP {} val PCKh {}'.format(epoch, val_loss, val_map, val_pckh))
            
            end_time = time.time()
            epoch_duration = end_time - start_time
            print(f'Epoch {epoch} completed in {epoch_duration:.2f} seconds')
            
            history["train_loss"].append(train_loss.numpy())
            history["val_loss"].append(val_loss.numpy())
            history["train_map"].append(train_map.numpy())
            history["val_map"].append(val_map.numpy())
            history["train_pckh"].append(train_pckh.numpy())
            history["val_pckh"].append(val_pckh.numpy())
            history["epoch_time"].append(epoch_duration)
            
            # save model when reach a new lowest validation loss
            if val_loss < self.lowest_val_loss:
                self.save_model(epoch, val_loss)
                self.lowest_val_loss = val_loss
            self.last_val_loss = val_loss
            
        return self.best_model, history

    def save_model(self, epoch, loss):
        model_name = MODEL_PATH + '/i_model-epoch-{}-loss-{:.4f}.h5'.format(epoch, loss)
        self.model.save_weights(model_name)
        self.best_model = model_name
        print("Model {} saved.".format(model_name))

In [11]:
IMAGE_SHAPE = (256, 256, 3)
HEATMAP_SIZE = (64, 64)

def create_dataset(tfrecords, batch_size, num_heatmap, is_train):
    preprocess = Preprocessor(
        IMAGE_SHAPE, (HEATMAP_SIZE[0], HEATMAP_SIZE[1], num_heatmap), is_train)

    dataset = tf.data.Dataset.list_files(tfrecords)
    dataset = tf.data.TFRecordDataset(dataset)
    dataset = dataset.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if is_train:
        dataset = dataset.shuffle(batch_size)

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [12]:
def train(epochs, learning_rate, num_heatmap, batch_size, train_tfrecords, val_tfrecords):
    strategy = tf.distribute.MirroredStrategy()
    global_batch_size = strategy.num_replicas_in_sync * batch_size
    train_dataset = create_dataset(
        train_tfrecords, global_batch_size, num_heatmap, is_train=True)
    val_dataset = create_dataset(
        val_tfrecords, global_batch_size, num_heatmap, is_train=False)

    if not os.path.exists(MODEL_PATH):
        os.makedirs(MODEL_PATH)

    with strategy.scope():
        train_dist_dataset = strategy.experimental_distribute_dataset(
            train_dataset)
        val_dist_dataset = strategy.experimental_distribute_dataset(
            val_dataset)

        model = StackedHourglassNetwork(IMAGE_SHAPE, 4, 1, num_heatmap)

        trainer = Trainer(
            model,
            epochs,
            global_batch_size,
            strategy,
            initial_learning_rate=learning_rate)

        print('Start training...')
        return trainer.run(train_dist_dataset, val_dist_dataset)

In [ ]:
train_tfrecords = os.path.join(TFRECORD_PATH, 'train*')
val_tfrecords = os.path.join(TFRECORD_PATH, 'val*')
epochs = 10
batch_size = 8
num_heatmap = 16
learning_rate = 0.0007

best_model_file, history = train(epochs, learning_rate, num_heatmap, batch_size, train_tfrecords, val_tfrecords)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/

/opt/conda/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py:374: UserWarning: To make it possible to preserve tf.data options across serialization boundaries, their implementation has moved to be part of the TensorFlow graph. As a consequence, the options value is in general no longer known at graph construction time. Invoking this method in graph mode retains the legacy behavior of the original implementation, but note that the returned value might not reflect the actual value of the options.
  warnings.warn("To make it possible to preserve tf.data options across "


Start distributed traininng...
Trained batch 1 batch loss 4.7368145 batch mAP 0.873046875 batch PCKh 0.033203125
Trained batch 2 batch loss 5.12859917 batch mAP 0.8046875 batch PCKh 0.021484375
Trained batch 3 batch loss 4.71795177 batch mAP 0.69140625 batch PCKh 0.048828125
Trained batch 4 batch loss 4.24303865 batch mAP 0.701171875 batch PCKh 0.068359375
Trained batch 5 batch loss 4.71073151 batch mAP 0.72265625 batch PCKh 0.103515625
Trained batch 6 batch loss 4.80445242 batch mAP 0.791015625 batch PCKh 0.033203125
Trained batch 7 batch loss 4.88720226 batch mAP 0.80078125 batch PCKh 0.0390625
Trained batch 8 batch loss 4.62131119 batch mAP 0.716796875 batch PCKh 0.0859375
Trained batch 9 batch loss 4.78496122 batch mAP 0.625 batch PCKh 0.0625
Trained batch 10 batch loss 4.07065535 batch mAP 0.63671875 batch PCKh 0.03125
Trained batch 11 batch loss 3.25651121 batch mAP 0.62890625 batch PCKh 0.001953125
Trained batch 12 batch loss 3.77517891 batch mAP 0.837890625 batch PCKh 0.0078125

Trained batch 101 batch loss 3.51722503 batch mAP 0.8515625 batch PCKh 0.29296875
Trained batch 102 batch loss 3.33230734 batch mAP 0.7578125 batch PCKh 0.34765625
Trained batch 103 batch loss 3.42952394 batch mAP 0.638671875 batch PCKh 0.15625
Trained batch 104 batch loss 3.35359669 batch mAP 0.77734375 batch PCKh 0.212890625
Trained batch 105 batch loss 3.61653471 batch mAP 0.84765625 batch PCKh 0.25
Trained batch 106 batch loss 3.56384754 batch mAP 0.892578125 batch PCKh 0.125
Trained batch 107 batch loss 3.58249021 batch mAP 0.8359375 batch PCKh 0.1796875
Trained batch 108 batch loss 3.25851107 batch mAP 0.76171875 batch PCKh 0.16796875
Trained batch 109 batch loss 3.62344 batch mAP 0.759765625 batch PCKh 0.193359375
Trained batch 110 batch loss 3.59533882 batch mAP 0.857421875 batch PCKh 0.166015625
Trained batch 111 batch loss 3.36576676 batch mAP 0.828125 batch PCKh 0.26953125
Trained batch 112 batch loss 3.30402255 batch mAP 0.748046875 batch PCKh 0.244140625
Trained batch 113 

Trained batch 201 batch loss 3.48237729 batch mAP 0.798828125 batch PCKh 0.294921875
Trained batch 202 batch loss 3.43884134 batch mAP 0.787109375 batch PCKh 0.25
Trained batch 203 batch loss 3.31623721 batch mAP 0.771484375 batch PCKh 0.2578125
Trained batch 204 batch loss 3.48128843 batch mAP 0.646484375 batch PCKh 0.287109375
Trained batch 205 batch loss 3.53599358 batch mAP 0.658203125 batch PCKh 0.259765625
Trained batch 206 batch loss 3.54750395 batch mAP 0.62109375 batch PCKh 0.30078125
Trained batch 207 batch loss 3.68402433 batch mAP 0.587890625 batch PCKh 0.19140625
Trained batch 208 batch loss 3.60266161 batch mAP 0.712890625 batch PCKh 0.1875
Trained batch 209 batch loss 3.33690143 batch mAP 0.70703125 batch PCKh 0.4140625
Trained batch 210 batch loss 3.56668329 batch mAP 0.740234375 batch PCKh 0.3125
Trained batch 211 batch loss 3.47925138 batch mAP 0.796875 batch PCKh 0.4453125
Trained batch 212 batch loss 3.10858 batch mAP 0.775390625 batch PCKh 0.41015625
Trained batch 

...

Trained batch 704 batch loss 2.64335012 batch mAP 0.08203125 batch PCKh 0.1953125
Trained batch 705 batch loss 2.59066558 batch mAP 0.06640625 batch PCKh 0.47265625
Trained batch 706 batch loss 2.86717319 batch mAP 0.111328125 batch PCKh 0.412109375
Trained batch 707 batch loss 2.42570591 batch mAP 0.083984375 batch PCKh 0.349609375
Trained batch 708 batch loss 2.67163467 batch mAP 0.099609375 batch PCKh 0.216796875
Trained batch 709 batch loss 2.48874521 batch mAP 0.095703125 batch PCKh 0.33984375
Trained batch 710 batch loss 2.89215708 batch mAP 0.052734375 batch PCKh 0.4765625
Trained batch 711 batch loss 2.78006864 batch mAP 0.07421875 batch PCKh 0.541015625
Trained batch 712 batch loss 2.81883955 batch mAP 0.05078125 batch PCKh 0.560546875
Trained batch 713 batch loss 2.69785047 batch mAP 0.10546875 batch PCKh 0.599609375
Trained batch 714 batch loss 2.82183695 batch mAP 0.087890625 batch PCKh 0.486328125
Trained batch 715 batch loss 2.74780464 batch mAP 0.095703125 batch PCKh 0.5

Trained batch 803 batch loss 2.92656684 batch mAP 0.078125 batch PCKh 0.4765625
Trained batch 804 batch loss 2.5482173 batch mAP 0.0546875 batch PCKh 0.654296875
Trained batch 805 batch loss 2.8256526 batch mAP 0.091796875 batch PCKh 0.50390625
Trained batch 806 batch loss 2.94492221 batch mAP 0.126953125 batch PCKh 0.5703125
Trained batch 807 batch loss 2.75895905 batch mAP 0.12109375 batch PCKh 0.521484375
Trained batch 808 batch loss 2.78356051 batch mAP 0.037109375 batch PCKh 0.5703125
Trained batch 809 batch loss 2.88292837 batch mAP 0.05859375 batch PCKh 0.501953125
Trained batch 810 batch loss 2.91985679 batch mAP 0.064453125 batch PCKh 0.5859375
Trained batch 811 batch loss 3.01891184 batch mAP 0.0546875 batch PCKh 0.490234375
Trained batch 812 batch loss 2.76263142 batch mAP 0.04296875 batch PCKh 0.544921875
Trained batch 813 batch loss 2.53713465 batch mAP 0.06640625 batch PCKh 0.546875
Trained batch 814 batch loss 2.64780402 batch mAP 0.046875 batch PCKh 0.232421875
Trained 

Trained batch 902 batch loss 2.41325116 batch mAP 0.0390625 batch PCKh 0.634765625
Trained batch 903 batch loss 2.57721853 batch mAP 0.095703125 batch PCKh 0.568359375
Trained batch 904 batch loss 2.53760934 batch mAP 0.091796875 batch PCKh 0.3046875
Trained batch 905 batch loss 2.69736362 batch mAP 0.080078125 batch PCKh 0.3203125
Trained batch 906 batch loss 2.67853951 batch mAP 0.046875 batch PCKh 0.37109375
Trained batch 907 batch loss 2.90854073 batch mAP 0.037109375 batch PCKh 0.419921875
Trained batch 908 batch loss 2.49895811 batch mAP 0.037109375 batch PCKh 0.19140625
Trained batch 909 batch loss 2.80843091 batch mAP 0.044921875 batch PCKh 0.353515625
Trained batch 910 batch loss 2.69339705 batch mAP 0.05078125 batch PCKh 0.533203125
Trained batch 911 batch loss 2.62442255 batch mAP 0.029296875 batch PCKh 0.611328125
Trained batch 912 batch loss 2.63627338 batch mAP 0.068359375 batch PCKh 0.626953125
Trained batch 913 batch loss 2.50267339 batch mAP 0.017578125 batch PCKh 0.64

Trained batch 1001 batch loss 2.62916684 batch mAP 0.01953125 batch PCKh 0.30078125
Trained batch 1002 batch loss 3.08875084 batch mAP 0.044921875 batch PCKh 0.443359375
Trained batch 1003 batch loss 3.34268355 batch mAP 0.04296875 batch PCKh 0.333984375
Trained batch 1004 batch loss 3.00923181 batch mAP 0.052734375 batch PCKh 0.328125
Trained batch 1005 batch loss 2.63348484 batch mAP 0.0859375 batch PCKh 0.486328125
Trained batch 1006 batch loss 2.46857643 batch mAP 0.015625 batch PCKh 0.412109375
Trained batch 1007 batch loss 2.53861237 batch mAP 0.017578125 batch PCKh 0.3046875
Trained batch 1008 batch loss 2.44291139 batch mAP 0.060546875 batch PCKh 0.591796875
Trained batch 1009 batch loss 2.53458667 batch mAP 0.0703125 batch PCKh 0.564453125
Trained batch 1010 batch loss 2.3807745 batch mAP 0.080078125 batch PCKh 0.564453125
Trained batch 1011 batch loss 2.42769265 batch mAP 0.078125 batch PCKh 0.560546875
Trained batch 1012 batch loss 2.75246572 batch mAP 0.0703125 batch PCKh 0

Trained batch 1099 batch loss 2.8629806 batch mAP 0.037109375 batch PCKh 0.5
Trained batch 1100 batch loss 2.87854838 batch mAP 0.080078125 batch PCKh 0.466796875
Trained batch 1101 batch loss 2.82697058 batch mAP 0.033203125 batch PCKh 0.54296875
Trained batch 1102 batch loss 2.98650599 batch mAP 0.076171875 batch PCKh 0.552734375
Trained batch 1103 batch loss 2.6116786 batch mAP 0.080078125 batch PCKh 0.240234375
Trained batch 1104 batch loss 2.53082156 batch mAP 0.0546875 batch PCKh 0.44140625
Trained batch 1105 batch loss 2.68861818 batch mAP 0.046875 batch PCKh 0.51171875
Trained batch 1106 batch loss 2.79067731 batch mAP 0.12109375 batch PCKh 0.548828125
Trained batch 1107 batch loss 2.79589128 batch mAP 0.13671875 batch PCKh 0.53515625
Trained batch 1108 batch loss 2.51815653 batch mAP 0.06640625 batch PCKh 0.556640625
Trained batch 1109 batch loss 2.86195517 batch mAP 0.064453125 batch PCKh 0.576171875
Trained batch 1110 batch loss 2.91607404 batch mAP 0.0546875 batch PCKh 0.38

In [ ]:
plt.figure(figsize=(14, 8))

plt.plot(history['train_loss'], 'r')
plt.plot(history['val_loss'], 'y')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))

plt.plot(history['train_map'], 'r')
plt.plot(history['val_map'], 'y')
plt.title('Model mAP')
plt.ylabel('mAP')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))

plt.plot(history['train_pckh'], 'r')
plt.plot(history['val_pckh'], 'y')
plt.title('Model PCKh')
plt.ylabel('PCKh')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='best')
plt.show()

In [ ]:
with open('history/history_sh_10.json', 'w') as f:
    pd.DataFrame(history).to_json(f)